In [1]:
# Import modules
from logger import setup_custom_logger
import sys
import pandas as pd    
import cbsodata
import datetime

In [2]:
# Setup of logger
try:
    logger = setup_custom_logger("H001_Total_Housing_Inventory")
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise


2021-03-18 13:42:03 INFO     H001_Total_Housing_Inventory starting


INFO:H001_Total_Housing_Inventory:starting


In [3]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-4
    currentYear = now.year-1
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise

2021-03-18 13:42:04 INFO     H001_Total_Housing_Inventory datetime loaded


INFO:H001_Total_Housing_Inventory:datetime loaded


In [4]:
# Dataset 82900NED
dataset_id = '82900NED'

# Table definitions

# StatusVanBewoning: T001235 = Totaal 
# StatusVanBewoning: A028725 = Bewoonde wonningen
# StatusVanBewoning: A028726 = Niet bewoonde wonningen
# RegioS: NL01 = Nederland
# TotaleWoningvoorraad_1 = Totaal(huur/koop)
# Koopwoningen_2 = totaal koopwoningen
# TotaalHuurwoningen_3 = totaal huurwoningen
# EigendomWoningcorporatie_4 = Eigendom woningcorperatie
# EigendomOverigeVerhuurders_5 = Eigendom overige verhuurders
# EigendomOnbekend_6 = Eigendom onbekend

In [17]:
try:
    logger.info(f'Retrieve data from dataset {dataset_id}')
    data = pd.DataFrame(cbsodata.get_data(
    f'{dataset_id}',
    filters=f"RegioS eq 'NL01  ' and substring(Perioden,0,4) ge '{yearMin}'",
    ))
except:
    logger.exception('error loading data from CBS Statline')
    raise

2021-03-18 13:44:11 INFO     H001_Total_Housing_Inventory Retrieve data from dataset 82900NED


INFO:H001_Total_Housing_Inventory:Retrieve data from dataset 82900NED


In [18]:
data

,ID,StatusVanBewoning,RegioS,Perioden,TotaleWoningvoorraad_1,Koopwoningen_2,TotaalHuurwoningen_3,EigendomWoningcorporatie_4,EigendomOverigeVerhuurders_5,EigendomOnbekend_6
0,5,Totaal,Nederland,2017,7686178,4321868,3283957,2273731,1010226,80353
1,6,Totaal,Nederland,2018,7740984,4386769,3275965,2268383,1007582,78250
2,7,Totaal,Nederland,2019,7814912,4487894,3299639,2295414,1004225,27379
3,8,Totaal,Nederland,2020,7891786,4517921,3342018,2294219,1047799,31847
4,4442,Bewoonde woningen,Nederland,2017,7331144,4189012,3102723,2211578,891145,39409
5,4443,Bewoonde woningen,Nederland,2018,7398054,4260602,3095275,2207510,887765,42177
6,4444,Bewoonde woningen,Nederland,2019,7469356,4348316,3102561,2225000,877561,18479
7,4445,Bewoonde woningen,Nederland,2020,7547587,4380084,3145536,2224790,920746,21967
8,8879,Niet bewoonde woningen,Nederland,2017,355034,132856,181234,62153,119081,40944
9,8880,Niet bewoonde woningen,Nederland,2018,342930,126167,180690,60873,119817,36073


In [3]:
# convert column "a" of a DataFrame
df["Perioden"] = pd.to_numeric(df["Perioden"])
df = df[(df.RegioS == "Nederland")]

In [4]:
# We parse our dataFrame to an Excel file that we can then load into Qlik Sense
df.to_excel("output.xlsx",
             sheet_name='Sheet_name_1') 

<h1><b>Data evaluation

<h3><b>Modeling, Evaluation, Deployment in qliksense